# Prerequistes

In [4]:
repo_id = "rajeshradhakrishnan/ml-news-classify-fastai"
#repo_id = "fastai X Hugging Face Group 2022/ml-news-classify"

In [13]:
# Installing from main and specifying ["fastai"] to automatically install the relevant packages and versions
# %%capture
!pip install -Uqq huggingface_hub["fastai"] datasets
# not needed just for reference:
#!pip install -Uqq transformers transformers['sentencepiece'] datasets

     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 362 kB 65.1 MB/s 
     |████████████████████████████████| 212 kB 74.0 MB/s 
     |████████████████████████████████| 140 kB 75.6 MB/s 
     |████████████████████████████████| 596 kB 63.0 MB/s 
     |████████████████████████████████| 127 kB 59.1 MB/s 
     |████████████████████████████████| 60 kB 5.5 MB/s 
     |████████████████████████████████| 197 kB 68.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [14]:
%%capture
!git clone https://github.com/ohmeow/blurr.git
%cd blurr
!pip install -e ".[dev]"

In [15]:
import torch
import datasets
import transformers
from fastai.text.all import *

from blurr.text.data.all import *
from blurr.text.modeling.all import *

# Prep Data

In [9]:
raw_datasets = datasets.load_dataset("rajeshradhakrishnan/malayalam_news")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset malayalam_news downloaded and prepared to /root/.cache/huggingface/datasets/rajeshradhakrishnan___malayalam_news/malayalam_news/1.0.0/7d6fb88a050b410d95b4f750165a6f797581be76cc65793eb4c6b71ab9ca9b09. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
raw_datasets['train'].features["label"].names

['business', 'entertainment', 'sports', 'technology']

In [11]:
def addTargeCols(example):
    return {"business": 1 if example["label"] == 0 else 0,
            "entertainment": 1 if example["label"] == 1 else 0,
            "sports": 1 if example["label"] == 2 else 0,
            "technology": 1 if example["label"] == 3 else 0}

In [12]:
raw_datasets = raw_datasets.map(addTargeCols)

Parameter 'function'=<function addTargeCols at 0x7fa45ae013b0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/4800 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

In [13]:
raw_datasets['train'] = raw_datasets['train'].add_column("is_valid", [False] * len(raw_datasets['train']))
raw_datasets['test'] = raw_datasets['test'].add_column("is_valid", [True] * len(raw_datasets['test']))

In [14]:
raw_datasets['train'][:2]

{'business': [0, 0],
 'entertainment': [0, 0],
 'is_valid': [False, False],
 'label': [2, 2],
 'sports': [1, 1],
 'technology': [0, 0],
 'text': ['മത്സര പ്രതിഫലമായി സ്വന്തമാക്കിയത് പതിനേഴ് ദശലക്ഷം ഡോളർ . ബാക്കി തുകയത്രയും പരസ്യ വരുമാനത്തിലൂടെ സ്വന്തമാക്കിയതാണ് . ബോൾട്ടിനോളം വിപണിമൂല്യമുള്ള മറ്റൊരു അത്ലറ്റില്ല . ഇതുകൊണ്ടു തന്നെയാണ് ബോൾട്ട് ലോകത്തെ ഏറ്റവും വരുമാനമുള്ള 100 കായിക താരങ്ങളുടെ പട്ടികയിലെ ഏക അത്ലറ്റായത് . പട്ടികയിൽ ഇരുപത്തിമൂന്നാം സ്ഥാനത്താണ് ജമൈക്കൻ ഇതിഹാസം . വിരമിച്ചതിന് ശേഷം ഫുട്ബോളിൽ സജീവമാകാൻ ഒരുങ്ങുന്ന വേഗരാജാവ് ഉസൈൻ ബോൾട്ട് ഫൌണ്ടേഷനും തുടക്കമിട്ടുകഴിഞ്ഞു . പാവപ്പെട്ട കുട്ടികളുടെ വിദ്യാഭ്യാസത്തിനാണ് ഫൌണ്ടേഷൻ പ്രധാനമായും സഹായിക്കുക . എട്ട് ഒളിംപിക് സ്വർണത്തിന് ഉടമയായ ബോൾട്ടിന്റെ പേരിലാണ് 100,200 മീറ്ററുകളിലെ ലോകറെക്കോർഡ് .',
  'ഇന്ത്യൻ പ്രീമിയർ ലീഗിൽ കൊൽക്കത്ത നൈറ്റ് റൈഡേഴ്സിനെതിരേ സൺറൈസേഴ്സിന് 140 റൺസ് വിജയലക്ഷ്യം . ടോസ് നഷ്ടപ്പെട്ട് ആദ്യ ബാറ്റിങ് ആരംഭിച്ച കൊൽത്തക്കയ്ക്ക് എട്ട് വിക്കറ്റ് നഷ്ടത്തിൽ 139 റൺസ് മാത്രമാണ് എടുക്കാൻ സാധിച്ചത് . 49 റൺസെടുത്ത ക്രിസ് ലിന്നാണ് അവരുട

In [15]:
df = raw_datasets.filter(lambda x: x["entertainment"] == 0)
print(df.num_rows)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'train': 3578, 'validation': 455, 'test': 467}


In [16]:
final_ds = datasets.concatenate_datasets([raw_datasets['train'].shuffle().select(range(1000))
                                        , raw_datasets['test'].shuffle().select(range(200))])
mlnews_df = pd.DataFrame(final_ds)
mlnews_df.head()

,text,label,business,entertainment,sports,technology,is_valid
0,"ആഷസ് പരമ്പരക്കായി ഓസ്ട്രേലിയയിലെത്തുന്ന ഇംഗ്ലണ്ട് ടീമിന് ശക്തമായ മുന്നറിയിപ്പുമായി ഓസ്ട്രേലിയൻ പേസർ മിച്ചൽ സ്റ്റാർക്ക് . പരിക്ക് മൂലം നാലു മാസമായി ക്രിക്കറ്റിൽ നിന്ന് വിട്ടുനിൽക്കുന്ന സ്റ്റാർക്ക് ഷെഫീൽഡ് ഷീൽഡിൽ സൌത്ത് വെയിൽസിനായി ഒരു മത്സരത്തിൽ രണ്ട് ഹാട്രിക്ക് നേടി ചരിത്രം കുറിച്ചു . വെസ്റ്റേൺ ഓസ്ട്രേലിയക്കെതിരെ ആയിരുന്നു സ്റ്റാർക്കിന്റെ തീ പാറുന്ന ബൌളിംഗ് . വെസ്റ്റേൺ ഓസ്ട്രേലിയയുടെ വാലറ്റക്കാരായ ജേസൺ ബെഹൻഡ്രോഫ് , ഡേവിഡ് മൂഡി , സൈമൺ മാക്കിൻ എന്നിവരെ വീഴ്ത്തിയാണ് സ്റ്റാർക്ക് ആദ്യ ഹാട്രിക്ക് തികച്ചത് . ആദ്യ ഇന്നിംഗ്സിൽ ഹാട്രിക്ക് അടക്കം നാലു വിക്കറ്റാണ് സ്റ്റാർക്ക് വീഴ്ത്തിയത് . രണ്ടാം ഇന്ന...",2,0,0,1,0,False
1,"രാജ്യത്ത് ഇന്ധനവില കുത്തനെ ഉയരുന്നു . പെട്രോൾ 21 പൈസയും , ഡീസൽ വില 22 പൈസയും കൂടി . സംസ്ഥാനത്തെ ഇന്നത്തെ ഇന്ധനവില തിരുവനന്തപുരത്ത് പെട്രോൾ വില 82രൂപ 81 പൈസ തിരുവനന്തപുരത്ത് ഡീസൽ വില 76 രൂപ 63 പൈസ കൊച്ചിയിൽ പെട്രോൾ വില 81 രൂപ 47 പൈസ കൊച്ചിയിലെ ഡീസൽ വില 75 രൂപ 38 പൈസ കോഴിക്കോട് പെട്രോൾ വില 81 രൂപ 72 പൈസ കോഴിക്കോട് ഡീസൽ വില 75 രൂപ 64 പൈസ",0,1,0,0,0,False
2,ദില്ലി ; അക്കൌണ്ടുകളിൽ മിനിമം ബാലൻസ് നിലനിർത്തണമെന്ന് നിശ്ചയിച്ചിതിന്റെ കാരണം വ്യക്തമാക്കി എസ്ബിഐ ചെയർ പേഴ്സൺ അരുന്ധതി ഭട്ടാചാര്യ . എടിഎം പ്രവർത്തനത്തിലുള്ള ചെലവുംജൻധൻ അക്കൌണ്ടുകൾ കൈകാര്യം ചെയ്യുന്നതിലുണ്ടായ നഷ്ടത്തിനും നികത്തനാണ് അക്കൌണ്ടുകളിൽ മിനിമമം ബാലൻസ് നിർബന്ധമാക്കിയതെന്ന് അരുന്ധതി പറഞ്ഞു . സ്റ്റേറ്റ് ബാങ്ക് ഓഫ് ഇന്ത്യയുടെ അക്കൌണ്ടുകളിൽ മിനിമം ബാലൻസ് തുക നിലനിർത്താത്തവർക്ക് പിഴ ചുമത്താനാണ് തീരുമാനം . പുതിയ നിർദ്ദേശം എസ്ബിഐ അംഗീകരിക്കുകെയും ചെയ്തു . നഗര - ഗ്രാമങ്ങൾ തരംതിരിച്ചാണ് മിനിമം ബാലൻസ് തുക നിശ്ചയിച്ചിരിക്കുന്നത് . മെട്രോ നഗരങ്ങളിൽ 5000 രൂപയും നഗരപ്രദേശങ്ങളിൽ 5000 രൂപയും അർധനഗര...,0,1,0,0,0,False
3,എക്സറേ ഫിലുമുകളിലൂടെ സൂര്യഗ്രഹണം കണ്ടിരുന്ന കാലം അവസാനിക്കുന്നു . ആഗസ്റ്റ് 21നു നടക്കുന്ന പൂർണ്ണ സൂര്യഗ്രഹണത്തിൻറെ ബഹിരാകാശ ദൃശ്യങ്ങൾ തൽസമയം ഇൻറർനെറ്റിലൂടെ കാണാം . യുഎസിലെ വിദ്യാർത്ഥികളുടെ സഹായത്തോടെ നാസ ഇതിനായി ഹൈ ആൾട്ടിട്യൂഡ് ബലൂണുകൾ ബഹിരാകാശത്തേക്ക് അയയ്ക്കും . ആദ്യമായാണ് ചന്ദ്രൻ സൂര്യനും ഭൂമിക്കും ഇടയിൽ വരുന്ന അപൂർവ നിമിഷങ്ങളാണ് ഇത്തരത്തിൽ പകർത്തുന്നത് . ഭൂമിക്ക് പുറത്തുള്ള ജീവൻറെ സാന്നിധ്യം പഠിക്കാനും നാസ ഹൈ ആൾട്ടിട്യൂഡ് ബലൂണുകൾ പ്രയോജനപ്പെടുത്തും . ഇൻറർനെറ്റിൽ തൽസമയം സംപ്രേഷണം ചെയ്യുന്നതിലൂടെ ഭൂമിയിലെ എല്ലാവർക്കും ഒരേ സമയം സൂര്യഗ്രഹണം കാണാനാകുമെന്ന് മൊണ്ടാന സ്റ്റേറ്റ് സർവ്വകലാശാലയിലെ...,3,0,0,0,1,False
4,ഏറെക്കാലം വാർത്തകളിൽ നിറഞ്ഞുനിന്ന് ഒടുവിൽ സംവിധായകൻ പ്രിയദർശൻ പ്രഖ്യാപിച്ച സിനിമയാണ് മരക്കാർ - അറബിക്കടലിന്റെ സിംഹം . മോഹൻലാൽ കുഞ്ഞാലിമരയ്ക്കാറായി എത്തുന്നുവെന്ന വാർത്ത പുറത്തുവന്നതോടെ ആരാധകരും ആകാംക്ഷയിലാണ് . ആരൊക്കെയാകും കുഞ്ഞാലി മരയ്ക്കാറിനൊപ്പം ഉണ്ടായിരിക്കും എന്നറിയാൻ . ചിത്രത്തിൽ മധുവും ഒരു പ്രധാന കഥാപാത്രത്തെ അവതരിപ്പിക്കുന്നുണ്ടെന്ന് അണിയറപ്രവർത്തകർ അറിയിച്ചു . നാല് മരയ്ക്കാന്മാരിൽ കുഞ്ഞാലി ഒന്നാമനായാണ് മധു അഭിനയിക്കുന്നത് . ആശിർവാദ് സിനിമാസിനൊപ്പം കോൺഫിഡൻറ് ഗ്രൂപ്പും മൂൺഷോട്ട് എൻറർടെയ്ൻമെൻറും സംയുക്തമായാണ് നിർമ്മാണം . ആശിർവാദിൻറെ 25 - ാം നിർമ്മാണസംരംഭമാണിത് . 100 കോടിയാണ് ബജറ്റ് ....,1,0,1,0,0,False


# Using Blurr High Level API

In [18]:
lbl_cols = raw_datasets['train'].features["label"].names
lbl_cols

['business', 'entertainment', 'sports', 'technology']

In [21]:
from transformers import AutoConfig, AutoModelForSequenceClassification
model_cls = AutoModelForSequenceClassification

pretrained_model_name = "xlm-roberta-base"
config = AutoConfig.from_pretrained(pretrained_model_name)
config.num_labels = len(lbl_cols)

hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(pretrained_model_name, model_cls=model_cls, config=config)
hf_model.config.problem_type = "multi_label_classification"

print(hf_arch)
print(type(hf_config))
print(type(hf_tokenizer))
print(type(hf_model))

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

xlm_roberta
<class 'transformers.models.xlm_roberta.configuration_xlm_roberta.XLMRobertaConfig'>
<class 'transformers.models.xlm_roberta.tokenization_xlm_roberta_fast.XLMRobertaTokenizerFast'>
<class 'transformers.models.xlm_roberta.modeling_xlm_roberta.XLMRobertaForSequenceClassification'>


In [22]:
learn = BlearnerForSequenceClassification.from_data(
    mlnews_df, pretrained_model_name, text_attr="text", label_attr=lbl_cols, dl_kwargs={"bs": 4}
)

learn.loss_func.thresh = 0.02

In [23]:
learn.fit_one_cycle(1, lr_max=1e-2)

epoch,train_loss,valid_loss,f1_score,accuracy_multi,time
0,0.630004,0.562437,0.000000,0.750000,00:52


In [33]:
learn.loss_func.thresh = 0.01

comment = """
2032 ഒളിമ്പിക്‌സിന് ബ്രിസ്‌ബെയ്ന്‍ വേദിയാകും; ഗെയിംസിന് വേദിയാകുന്ന മൂന്നാമത്തെ ഓസ്‌ട്രേലിയന്‍ നഗരം
"""
learn.predict(comment)

[{'class_indices': [1, 1, 1, 1],
  'class_labels': ['business', 'entertainment', 'sports', 'technology'],
  'labels': ['business', 'entertainment', 'sports', 'technology'],
  'probs': [0.22391068935394287,
   0.2475249022245407,
   0.2982001304626465,
   0.2526259124279022],
  'scores': [0.22391068935394287,
   0.2475249022245407,
   0.2982001304626465,
   0.2526259124279022]}]

# Share in Huggingface

In [24]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [32]:
from huggingface_hub import push_to_hub_fastai

push_to_hub_fastai(learner=learn, repo_id=repo_id)

Cloning https://huggingface.co/rajeshradhakrishnan/ml-news-classify-fastai into local empty directory.
Adding files tracked by Git LFS: ['model.pkl']. This may take a bit of time if the files are large.


Upload file model.pkl:   0%|          | 3.33k/1.04G [00:00<?, ?B/s]

To https://huggingface.co/rajeshradhakrishnan/ml-news-classify-fastai
   bfea4f0..bbe4199  main -> main



'https://huggingface.co/rajeshradhakrishnan/ml-news-classify-fastai/commit/bbe41991bc2c4e045ed50e0e7321ad5b4fd5530f'

# Loading a Learner from the Hugging Face Hub

In [16]:
from huggingface_hub import from_pretrained_fastai
# ഞാൻ ചിന്തിക്കുന്ന കുറച്ചു ആശയങ്ങൾ പ്രാവർത്തികം ആക്കുക എന്നുള്ളതാണ്
# ഇതുവരെ ഞാൻ കണ്ടുമുട്ടിയതും പരിചയവുമുള്ള സോഫ്റ്റ്‌വെയർ സേവനങ്ങൾ 
# എങ്ങനെ പ്രയോജനപ്പെടുത്താം എന്നുള്ള വിവരങ്ങൾ ഇവിടെ പരിചയ പെടുത്തുന്നതുമാണ്. 
learner = from_pretrained_fastai(repo_id)

Downloading:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/843 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [26]:
sentences = ["ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം",
             "വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം"]

probs = learner.predict(sentences)
# 'business', 'entertainment', 'sports', 'technology'
for idx in range(len(sentences)):
  print(f"Probability that sentence '{sentences[idx]}' is business is: {100*probs[idx]['probs'][0]:.2f}%")
  print(f"Probability that sentence '{sentences[idx]}' is entertainment is: {100*probs[idx]['probs'][1]:.2f}%")
  print(f"Probability that sentence '{sentences[idx]}' is sports is: {100*probs[idx]['probs'][2]:.2f}%")
  print(f"Probability that sentence '{sentences[idx]}' is technology is: {100*probs[idx]['probs'][3]:.2f}%")

Probability that sentence 'ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം' is business is: 22.39%
Probability that sentence 'ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം' is entertainment is: 24.75%
Probability that sentence 'ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം' is sports is: 29.82%
Probability that sentence 'ഓഹരി വിപണി തകരുമ്പോള്‍ നിക്ഷേപം എങ്ങനെ സുരക്ഷിതമാക്കാം' is technology is: 25.26%
Probability that sentence 'വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം' is business is: 22.39%
Probability that sentence 'വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം' is entertainment is: 24.75%
Probability that sentence 'വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം' is sports is: 29.82%
Probability that sentence 'വാര്‍ണറുടെ ഒറ്റക്കയ്യന്‍ ക്യാച്ചില്‍ അമ്പരന്ന് ക്രിക്കറ്റ് ലോകം' is technology is: 25.26%


# Create a Gradio demo to showcase your model

In [2]:
!pip install -Uqq gradio

     |████████████████████████████████| 5.1 MB 31.5 MB/s 
     |████████████████████████████████| 256 kB 57.4 MB/s 
     |████████████████████████████████| 54 kB 4.0 MB/s 
     |████████████████████████████████| 1.1 MB 57.9 MB/s 
     |████████████████████████████████| 53 kB 2.9 MB/s 
     |████████████████████████████████| 2.0 MB 54.7 MB/s 
     |████████████████████████████████| 84 kB 4.6 MB/s 
     |████████████████████████████████| 212 kB 72.1 MB/s 
     |████████████████████████████████| 271 kB 74.1 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 144 kB 71.6 MB/s 
     |████████████████████████████████| 11.1 MB 63.9 MB/s 
     |████████████████████████████████| 63 kB 2.7 MB/s 
     |████████████████████████████████| 80 kB 11.1 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 4.0 MB 62.4 MB/s 
     |████████████████████████████████| 856 kB 58.9 MB/s 
     |████████████████████

In [27]:
import gradio as gr

description = "Malayalam news classifier with BERT"
title = "Classify malayalam news feeds"
examples = [["ഞാൻ ചിന്തിക്കുന്ന കുറച്ചു ആശയങ്ങൾ"],
            ["ഇതുവരെ ഞാൻ കണ്ടുമുട്ടിയതും പരിചയവുമുള്ള"]]

# rajeshradhakrishnan/malayalam_news_classifier
# repoid = rajeshradhakrishnan/ml-news-classify-fastai
interface = gr.Interface.load("huggingface/" + repo_id,
            description=description,
            examples=examples
)

interface.launch()


Fetching model from: https://huggingface.co/rajeshradhakrishnan/ml-news-classify-fastai


ValueError: ignored

In [9]:
gr.Interface.load("spaces/rajeshradhakrishnan/malayalam-news-classify").launch()

Fetching interface from: https://huggingface.co/spaces/rajeshradhakrishnan/malayalam-news-classify


TypeError: ignored

---
tags:
- fastai
---

# Amazing!

🥳 Congratulations on hosting your fastai model on the Hugging Face Hub!

# Some next steps
1. Fill out this model card with more information (see the template below and the [documentation here](https://huggingface.co/docs/hub/model-repos))!

2. Create a demo in Gradio or Streamlit using 🤗 Spaces ([documentation here](https://huggingface.co/docs/hub/spaces)).

3. Join the fastai community on the [Fastai Discord](https://discord.com/invite/YKrxeNn)!

Greetings fellow fastlearner 🤝! Don't forget to delete this content from your model card.


---


# Model card

## Model description
More information needed

## Intended uses & limitations
More information needed

## Training and evaluation data
More information needed
